In [1]:
from pynq.overlays.base import BaseOverlay
from IPython.display import clear_output
from pynq import allocate
from time import sleep
import plotly.graph_objs as go
import ipywidgets as ipw
import numpy as np
import threading
import socket
import sys

overlay = BaseOverlay('base.bit')

UDP_IP = "10.42.0.1" #PC-10.42.0.1 (enx0050b623bd36) RFSoC-10.42.0.85 General-127.0.0.1
UDP_PORT = 5005

In [2]:
for led in overlay.leds:
    led.off()

overlay.init_rf_clks()

In [3]:
def updRX():
    rx = 1;
    sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
    sock.bind((UDP_IP, UDP_PORT))

    while rx:
        data, addr = sock.recvfrom(4096) # buffer size in bytes
        print("\nreceived message: %s" % data)
        rx = 0

def updTX(message):    
    print("UDP target IP: %s" % UDP_IP)
    print("UDP target port: %s" % UDP_PORT)
    print("send message: %s" % message)

    sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
    sock.sendto(message, (UDP_IP, UDP_PORT))

In [4]:
def txSamples(num_dac, num_adc):
    dac= overlay.radio.transmitter.channel[num_dac]
    dac.dac_block.MixerSettings['Freq'] = 1000 # MHz 
    dac.control.gain = 0.5
    dac.control.enable = True
    
    adc = overlay.radio.receiver.channel[num_adc]
    delayLEDs = 0.1

    for led in overlay.leds:
        led.on()
    while (overlay.buttons[0].read()==0):
        if (overlay.buttons[3].read()):
            clear_output(wait=True)
            power = overlay.switches.read()+4;
            number_samples = 2**power
            print("Power value: {0}".format(power))
            if (power > 15):
                print("Packet size incorrect, power value should be in range 4 to 15.")
            else:
                for led in overlay.leds:
                    led.off()
                sleep(delayLEDs)
                for led in overlay.leds:
                    led.toggle()
                    sleep(delayLEDs)
                updTX(np.array(adc.transferIQ(number_samples))) #np.array per passar de list a array

    clear_output(wait=True)
    print('End of connection...')
    for led in overlay.leds:
        led.off()

In [5]:
ttxSamples = threading.Thread(target=txSamples,args=(1,3))
    #tRX = threading.Thread(target=updRX)

ttxSamples.start()
    #tRX.start()

In [ ]:
ttxSamples.join()
    #tRX.join()


#Si error OSError: [Errno 90] Message too long:
#
    #You can do that by changing the MTU (Maximum Transfer Unit) of your interface. 
    #This is the maximal number of bytes a frame can contain.
    #
    #ifconfig eth0 mtu 2000 up
    
#Ara per ara (20-10-23), es queixa quan són més de "Power value: 13"

Power value: 12
UDP target IP: 10.42.0.1
UDP target port: 5005
send message: [-4911  -248 -4225 ...  3777  -545  4883]
